In [ ]:
!pip install datasets transformers[sentencepiece] sacrebleu -q
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 4.1 MB/s eta 0:00:00


In [ ]:
import os
import sys
import transformers
import tensorflow as tf
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import TFAutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import AdamWeightDecay
from transformers import AutoTokenizer, TFAutoModelForSeq2SeqLM
import pandas as pd
import numpy as np
import json
import kagglehub

In [ ]:
model_checkpoint = "Helsinki-NLP/opus-mt-en-hi"

In [ ]:
path = kagglehub.dataset_download("sayedshaun/english-to-bengali-for-machine-translation")

# Load dataset from the extracted directory
raw_datasets = load_dataset(path)


100%|██████████| 1.71M/1.71M [00:00<00:00, 3.07MB/s]

Extracting files...


Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['en', 'bn'],
        num_rows: 39065
    })
})

In [ ]:
local_path = kagglehub.dataset_download("sayedshaun/english-to-bengali-for-machine-translation")

# Load dataset from the downloaded folder
raw_datasets = load_dataset(local_path)  # Load from local path

# Convert dataset to a dictionary
dataset_dict = {split: raw_datasets[split].to_list() for split in raw_datasets}

# Save as JSON
json_path = "/content/raw_datasets.json"  # Save in Colab's file system
with open(json_path, "w", encoding="utf-8") as f:
    json.dump(dataset_dict, f, indent=4, ensure_ascii=False)

print(f"Dataset saved as JSON at: {json_path}")

Dataset saved as JSON at: /content/raw_datasets.json


In [ ]:
raw_datasets["train"][1]

{'en': 'a girl going into a wooden building .',
 'bn': 'একটি মেয়ে শিশু একটি কাঠের বাড়িতে ঢুকছে'}

In [ ]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['en', 'bn'],
        num_rows: 39065
    })
})

In [ ]:
raw_datasets['train'][1]


{'en': 'a girl going into a wooden building .',
 'bn': 'একটি মেয়ে শিশু একটি কাঠের বাড়িতে ঢুকছে'}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.39k [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/812k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [ ]:
tokenizer("hello, how are you man?")

{'input_ids': [39915, 2, 287, 54, 27, 259, 22, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
with tokenizer.as_target_tokenizer():
  print(tokenizer(["একটি মেয়ে শিশু একটি কাঠের বাড়িতে ঢুকছে"]))

{'input_ids': [[44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
print(raw_datasets["train"].column_names)


['en', 'bn']


In [ ]:
# preprocessing

max_input_length = 128
max_target_length = 128


def preprocess_function(examples):
    inputs = examples["en"]  # Source language (English)
    targets = examples["bn"]  # Target language (Bengali)
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [ ]:
preprocess_function(raw_datasets["train"][:2])

{'input_ids': [[19, 1075, 21, 19, 41065, 8290, 23, 24751, 213, 19, 488, 8, 25871, 21, 132, 4683, 284, 44, 3, 0], [19, 3821, 1098, 242, 19, 20274, 3370, 44, 3, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 40, 0], [44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 44, 1, 0]]}

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/39065 [00:00<?, ? examples/s]

In [ ]:
model = TFAutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

tf_model.h5:   0%|          | 0.00/306M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at Helsinki-NLP/opus-mt-en-hi.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [ ]:
batch_size = 32
learning_rate = 2e-5
weight_decay = 0.01
num_train_epochs = 500

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf")

In [ ]:
generation_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model, return_tensors="tf", pad_to_multiple_of=128)

In [ ]:
train_dataset = model.prepare_tf_dataset(
    tokenized_datasets["train"],
    batch_size=batch_size,
    shuffle=True,
    collate_fn=data_collator,
)

In [ ]:
print(raw_datasets.keys())


dict_keys(['train'])


In [ ]:
from datasets import DatasetDict

# Split dataset (90% train, 10% validation)
split_dataset = raw_datasets["train"].train_test_split(test_size=0.1)

# Create a new dataset dictionary
raw_datasets = DatasetDict({
    "train": split_dataset["train"],
    "validation": split_dataset["test"]  # Use test split as validation
})

# Check available splits
print(raw_datasets.keys())  # Should print: dict_keys(['train', 'validation'])


dict_keys(['train', 'validation'])


In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

validation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=batch_size,
    shuffle=False,
    collate_fn=data_collator,
)


Map:   0%|          | 0/35158 [00:00<?, ? examples/s]

Map:   0%|          | 0/3907 [00:00<?, ? examples/s]

In [ ]:
generation_dataset = model.prepare_tf_dataset(
    tokenized_datasets["validation"],
    batch_size=8,
    shuffle=False,
    collate_fn=generation_data_collator,
)

In [ ]:
optimizer = AdamWeightDecay(learning_rate=learning_rate, weight_decay_rate=weight_decay)
model.compile(optimizer=optimizer)

In [ ]:
# epochs is been replaced by 1
model.fit(train_dataset, validation_data=validation_dataset, epochs=1)

1220/1220 [==============================] - 21616s 18s/step - loss: 0.4078 - val_loss: 0.1698


In [ ]:
model.save_pretrained("tf_model/")

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:394: UserWarning: Some non-default generation parameters are set in the model config. These should go into either a) `model.generation_config` (as opposed to `model.config`); OR b) a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model).This warning will become an exception in the future.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61949]]}
  warnings.warn(


In [ ]:
model.save('/content/my_trained_model')  # Saves model as a directory


In [ ]:
from google.colab import files
import shutil

# Zip the model folder before downloading
shutil.make_archive('/content/my_trained_model', 'zip', '/content/my_trained_model')

# Download the zipped model
files.download('/content/my_trained_model.zip')





<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# ***Testing Model***

In [ ]:
# testing
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = TFAutoModelForSeq2SeqLM.from_pretrained("tf_model/")

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
All model checkpoint layers were used when initializing TFMarianMTModel.

All the layers of TFMarianMTModel were initialized from the model checkpoint at tf_model/.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFMarianMTModel for predictions without further training.


In [ ]:
input_text  = "i have myslef"

tokenized = tokenizer([input_text], return_tensors='np')
out = model.generate(**tokenized, max_length=128)
print(out)

tf.Tensor([[61949    44     1    44     1    44     1     0 61949 61949 61949 61949]], shape=(1, 12), dtype=int32)


In [ ]:
with tokenizer.as_target_tokenizer():
    print(tokenizer.decode(out[0], skip_special_tokens=True))